# Potencial Ornitológico Fueguino
### **Autor:** Pablo Jusim

# Script de creación de la base de datos para modelar

En este script se creará el dataframe con los datos para el modelado a partir de las observaciones obtenidas de iNaturalist y ebird. El preprocesamiento de ambas fuentes de datos se realizará de manera independiente en sendos notebooks.

## Importaciones

In [1]:
# Modulos de terceros
import pandas as pd
from pathlib import Path
import sys
from pyogrio.errors import DataSourceError

In [2]:
# Modulos propios
sys.path.append(str(Path('..')/'src'))

import grillado
import asociar_grilla
import utils

## Carga de datos crudos del proyecto

Dentro de la carpeta data/raw se encuentran los datos tal cual fueron obtenidos.

Los datos descargados fueron preprocesados según los script 01a-conversion_inat.ipynb y 01b-conversion_ebird.ipynb

### Crear grilla de Tierra del Fuego

In [3]:
# Verificar si la grilla ya fue creada, sino crearla
RUTA_CONTORNO = '../data/external/contorno_tdf.gpkg'
RUTA_GRILLA = Path('../data/raw/grilla_tdf_vacia.gpkg')

def crear_grilla(ruta, archivo_cont):
    if ruta.exists():
        print(f"La grilla ya existe en {ruta}.")
    else:
        geo_df = grillado.main(
            archivo_cont = archivo_cont,
            geoide = 4326,  # WGS 84
            lado_celda = 5  # tamaño de lado en kilometros
        )

        geo_df.to_file(RUTA_GRILLA, driver='GPKG',
                                layer='grilla_tdf_vacia')

try:
    crear_grilla(RUTA_GRILLA, RUTA_CONTORNO)
except DataSourceError:
    print("Error al abrir el archivo GeoPackage")


La grilla ya existe en ..\data\raw\grilla_tdf_vacia.gpkg.


### Unir los datos de eBird e iNaturalist

In [4]:
df_ebird = pd.read_csv('../data/raw/data_ebird.csv')
df_inat = pd.read_csv('../data/raw/data_inat.csv')
df_obs = pd.concat([df_ebird, df_inat])

# Revisar que la union es correcta viendo cuantas filas tiene cada dataframe
print(f'obs ebird: {len(df_ebird)}')
print(f'obs inat: {len(df_inat)}')
print(f'obs sumadas: {len(df_ebird) + len(df_inat)}')
print(f'obs totales: {len(df_obs)}')

obs ebird: 426837
obs inat: 8087
obs sumadas: 434924
obs totales: 434924


### Asociar cada registro la celda de la grilla

In [5]:
# Punto de partida: directorio actual (notebooks/)
BASE_DIR = Path.cwd()
# Subir un nivel al directorio raíz del proyecto
PROYECTO_DIR = BASE_DIR.parent
# Construir ruta completa al archivo GPKG
DIR_DATOS = PROYECTO_DIR / 'data' / 'raw'

nombre_grilla = 'grilla_tdf_vacia.gpkg'
ruta_grilla = DIR_DATOS / nombre_grilla

In [6]:
# Asociar grilla a los datos de observaciones

df_grillada =asociar_grilla.assign_grid_cell_ids(
    grilla = ruta_grilla,
    datos_georef = df_obs,
    grid_id_field = 'grid_id'
)

### Eliminar registros erróneos o raros

#### Eliminar registros de especies raras
Los registros de especies con pocas ocurrencias podrían ser por especies muy difíciles de observar, especies que alguna vez han alcanzado Tierra del Fuego por algún motivo o errores de registro e identificación. Ante la imposibilidad de distinguir estos casos, se eliminan los registros de especies con menos de 5 ocurrencias.

In [7]:
df_obs_com = utils.eliminar_spp_raras(
    data = df_grillada, 
    ocurrencia = 5
)
print(f'obs totales spp no raras: {len(df_obs_com)}')

obs totales spp no raras: 406793


#### Eliminar los registros erróneos
Se consideran erróneos los registros que cumplan alguna de las siguentes características:
- Contengan en el texto el símbolo "/" o "\", lo que indica duda entre más de una especie
- Contengan la palabra "sp." o "spp." que indican que se conoce el género pero no la especie.
- Tengan una cantidad de palabras diferente de 2 (género + epíteto) o 3 (género + epíteto + subespecie). Si tienen 3 palabras solo se conservan las 2 primeras

In [8]:
# 0) Limpiar whitespace
df_corr = df_obs_com.copy()
df_corr['scientific_name'] = df_corr['scientific_name'].str.strip()

# 1) Eliminar nombres con '/' o '\'
df_corr = df_corr[~df_corr['scientific_name']
                   .str.contains(r'[\\/]', regex=True)]

# 2) Eliminar 'sp.' o 'spp.' (insensible a mayúsculas)
df_corr = df_corr[~df_corr['scientific_name']
                   .str.contains(r'(?i)\b(?:sp\.|spp\.)', regex=True)]

print(f'obs corregidas: {len(df_corr)}')


obs corregidas: 404164


## Crear dataframe de registros por celda y por especie
Crear un dataframe donde cada fila sea una celda y cada columna sea una especie. El dataframe mostrará la cantidad de registros por celda y especie.

In [9]:
df_grilla_spp = utils.crear_df_conteos(df_corr)
df_grilla_spp.head()

scientific_name,grid_id,Anairetes parulus,Anarhynchus falklandicus,Anas bahamensis,Anas flavirostris,Anas georgica,Anser anser,Anthus correndera,Aphrastura spinicauda,Aptenodytes patagonicus,...,Tyrannus savana,Tyrannus tyrannus,Tyto furcata,Upucerthia dumetaria,Vanellus chilensis,Vultur gryphus,Zenaida auriculata,Zenaida meloda,Zonibyx modestus,Zonotrichia capensis
0,25,0,0,0,0,1,0,1,0,0,...,0,0,0,0,2,1,0,0,1,1
1,26,3,0,0,3,3,0,1,5,0,...,0,0,0,0,2,0,0,0,2,5
2,27,0,0,0,1,2,0,2,3,1,...,0,0,0,0,2,3,0,0,2,3
3,28,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,70,1,0,0,7,14,0,12,4,0,...,0,0,0,0,8,0,0,0,2,14


## Exportar el dataframe generado como un csv

In [10]:
df_grilla_spp.to_csv('../data/interim/grilla_tdf_spp.csv', index=False)